##Setup

You will need to make a copy of this notebook in your Google Drive before you can edit the homework files. You can do so with **File &rarr; Save a copy in Drive**.

In [1]:
#@title mount your Google Drive
#@markdown Your work will be stored in a folder called `cs285_f2020` by default to prevent Colab instance timeouts from deleting your edits.

import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
#@title set up mount symlink

import os
DRIVE_PATH = '/content/gdrive/MyDrive/homework_fall2020'
DRIVE_PYTHON_PATH = DRIVE_PATH.replace('\\', '')
if not os.path.exists(DRIVE_PYTHON_PATH):
  %mkdir $DRIVE_PATH

## the space in `My Drive` causes some issues,
## make a symlink to avoid this
SYM_PATH = '/content/cs285_f2020'
if not os.path.exists(SYM_PATH):
  !ln -s $DRIVE_PATH $SYM_PATH

In [ ]:
#@title apt install requirements

#@markdown Run each section with Shift+Enter

#@markdown Double-click on section headers to show code.

!apt update 
!apt install -y --no-install-recommends \
        build-essential \
        curl \
        git \
        gnupg2 \
        make \
        cmake \
        ffmpeg \
        swig \
        libz-dev \
        unzip \
        zlib1g-dev \
        libglfw3 \
        libglfw3-dev \
        libxrandr2 \
        libxinerama-dev \
        libxi6 \
        libxcursor-dev \
        libgl1-mesa-dev \
        libgl1-mesa-glx \
        libglew-dev \
        libosmesa6-dev \
        lsb-release \
        ack-grep \
        patchelf \
        wget \
        xpra \
        xserver-xorg-dev \
        xvfb \
        python-opengl \
        ffmpeg

In [ ]:
#@title clone homework repo

%cd $SYM_PATH

!!git clone https://github.com/berkeleydeeprlcourse/homework_fall2020.git
%cd homework_fall2020/hw1
%pip install -r requirements_colab.txt
%pip install -e .
!pip install roboschool==1.0.48
!pip install gym==0.15.4
!sudo apt-get install xvfb

In [ ]:
!pip install roboschool==1.0.48 gym==0.15.4

**PLEASE RESET THE SESSION**

**Note:You don't have to run above cells agian.**




In [1]:
#@title set up virtual display
!pip install  pyvirtualdisplay
from pyvirtualdisplay import Display

display = Display(visible=0, size=(1400, 900))
display.start()

In [7]:
#@title test virtual display
# %cd /content/cs285_f2020/homework_fall2020/hw1
#@markdown If you see a video of a four-legged ant fumbling about, setup is complete!
import roboschool
import gym
from cs285.infrastructure.colab_utils import (
    wrap_env,
    show_video
) 

env = wrap_env(gym.make("RoboschoolAnt-v1"))

observation = env.reset()
for i in range(100):
    env.render(mode='rgb_array')
    obs, rew, term, _ = env.step(env.action_space.sample() ) 
    if term:
      break;
            
env.close()
print('Loading video...')
show_video()

Loading video...


## Editing Code

To edit code, click the folder icon on the left menu. Navigate to the corresponding file (`cs285_f2020/...`). Double click a file to open an editor. There is a timeout of about ~12 hours with Colab while it is active (and less if you close your browser window). We sync your edits to Google Drive so that you won't lose your work in the event of an instance timeout, but you will need to re-mount your Google Drive and re-install packages with every new instance.

In [8]:
#@title imports
!pip install tensorboardX
import os
import time
import numpy as np
from cs285.infrastructure.rl_trainer import RL_Trainer
from cs285.agents.bc_agent import BCAgent
from cs285.policies.loaded_gaussian_policy import LoadedGaussianPolicy

%load_ext autoreload
%autoreload 2

**1.Behaviour Cloning**

In [13]:
%cd "/content/gdrive/MyDrive/homework_fall2020/hw1"
# !rm -r "/content/cs285_f2020/homework_fall2020/hw1/data"
!python /content/gdrive/MyDrive/homework_fall2020/hw1/run_hw1.py \
--expert_policy_file cs285/policies/experts/Ant.pkl \
--env_name RoboschoolAnt-v1 --exp_name bc_ant --n_iter 1 \
--expert_data cs285/expert_data/expert_data_Ant-v2.pkl \
--video_log_freq -1 

/content/gdrive/MyDrive/homework_fall2020/hw1
########################
logging outputs to  /content/gdrive/MyDrive/homework_fall2020/hw1/../../data/q1_bc_ant_RoboschoolAnt-v1_11-02-2021_07-19-02
########################
GPU not detected. Defaulting to CPU.
Sequential(
  (input): Linear(in_features=28, out_features=64, bias=True)
  (hidden__0): Linear(in_features=64, out_features=64, bias=True)
  (hidden_0): Tanh()
  (hidden__1): Linear(in_features=64, out_features=64, bias=True)
  (hidden_1): Tanh()
  (out): Linear(in_features=64, out_features=8, bias=True)
  (output): Identity()
)
Loading expert policy from... cs285/policies/experts/Ant.pkl
obs (1, 111) (1, 111)
Done restoring expert policy...


********** Iteration 0 ************


Training agent using sampled data from replay buffer...

Beginning logging procedure...

Eval_AverageReturn : 14.59140682220459
Eval_StdReturn : 16.70257568359375
Eval_MaxReturn : 57.218746185302734
Eval_MinReturn : 2.0141587257385254
Eval_AverageEpLen : 5

**2.DAgger**

In [ ]:
!python /content/gdrive/MyDrive/homework_fall2020/hw1/run_hw1.py \
--expert_policy_file cs285/policies/experts/Ant.pkl \
--env_name RoboschoolAnt-v1 --exp_name dagger_ant --n_iter 10 \
--do_dagger --expert_data cs285/expert_data/expert_data_Ant-v2.pkl \
--video_log_freq -1 